### Main funtion that takes in two matrics (3D and 2D) and return calculated params


For starting create a random matrix

In [1]:
import random 
import numpy as np

In [157]:
mW = np.loadtxt('myWCoords.txt')
mC = np.loadtxt('myCCoords.txt')
mW = np.append(mW,np.ones((mW.shape[0],1)),axis=1)


In [185]:
W = mW[:6,:]
C = mC[:6,:]
W.shape,C.shape

((6, 4), (6, 2))

Specify number of observation points 

In [186]:
n=6

Create 3d matrix which represents world coordinate system and 2d matrix which represents camera coordinate system
Note: We'll append 1 in world coordinates so dim will be (nx4) for 2d it will be (nx2)

In [3]:
W = np.array([[random.randint(1,10) for i in range(3)]+[1] for j in range(n)])
W

array([[ 5,  5,  7,  1],
       [ 3,  3,  9,  1],
       [ 4,  8,  9,  1],
       [ 9,  9,  7,  1],
       [ 9,  1, 10,  1],
       [ 1,  1,  1,  1]])

In [4]:
C = np.array([[random.randint(200,400) for i in range(2)] for j in range(n)])
C

array([[375, 306],
       [337, 302],
       [213, 263],
       [306, 396],
       [317, 319],
       [333, 254]])

Now we will make the P matrix (from P.m = 0) which has dim (2nX12)

Now fill in the value according to the eqaution:    
Pi x m1 + 0 x m2 -  X0 x Pi x m3 = 0    
0 x m1 + Pi x m2 -  Y0 x Pi x m3 = 0

In [187]:
P = np.zeros((2*n,12))
for i in range(n):
    P[i*2,0:4] = W[i]
    P[i*2,8:12] = -C[i][0]*W[i]
    P[i*2+1,4:8] = W[i]
    P[i*2+1,8:12] = -C[i][1]*W[i]
P

array([[ 0.000e+00,  0.000e+00,  0.000e+00,  1.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00, -0.000e+00, -0.000e+00,
        -0.000e+00, -1.570e+02],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  1.000e+00, -0.000e+00, -0.000e+00,
        -0.000e+00, -1.250e+02],
       [ 0.000e+00,  1.000e+00,  0.000e+00,  1.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00, -0.000e+00, -1.540e+02,
        -0.000e+00, -1.540e+02],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         1.000e+00,  0.000e+00,  1.000e+00, -0.000e+00, -1.740e+02,
        -0.000e+00, -1.740e+02],
       [ 0.000e+00,  2.000e+00,  0.000e+00,  1.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00, -0.000e+00, -3.040e+02,
        -0.000e+00, -1.520e+02],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         2.000e+00,  0.000e+00,  1.000e+00, -0.000e+00, -4.500e+02,
        -0.000e+00,

Now we will go on calculating SVD for Pt x P  matrix. Then choosing the Eigenvector corresponding to minimum eigen value to obtain the solution P x m  = 0  

In [188]:
u, s, vh = np.linalg.svd(np.matmul(P.T,P))
s.shape,vh.shape

((12,), (12, 12))

In [189]:
m = vh[np.argmin(s)]
m.shape,m

((12,), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.]))

In [190]:
m = m.reshape((3,4))
m.shape,m

((3, 4), array([[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  0.]]))

In [191]:
a1 = m[0,0:3]
a2 = m[1,0:3]
a3 = m[2,0:3]
B = m[:,-1]
a1,a2,a3,B

(array([0., 0., 0.]),
 array([0., 0., 0.]),
 array([ 0.,  0., -1.]),
 array([0., 0., 0.]))

In [192]:
ro = 1/np.linalg.norm(a3)
r3 = ro*a3
X0 = ro*ro*np.dot(a1, a3);
Y0 = ro*ro*np.dot(a2, a3);
alpha = ro*ro*np.linalg.norm(np.cross(a1, a3));
beta = ro*ro*np.linalg.norm(np.cross(a2, a3));

In [193]:
ro,r3,X0,Y0,alpha,beta

(1.0, array([ 0.,  0., -1.]), 0.0, 0.0, 0.0, 0.0)

In [194]:
cosTheta = -np.dot(np.cross(a1,a3),np.cross(a2,a3))/(np.linalg.norm(np.cross(a1,a3))*np.linalg.norm(np.cross(a2,a3)))
sinTheta = (1-cosTheta**2)**(1/2)
r1 = np.cross(a2,a3)/np.linalg.norm(np.cross(a2,a3))
r2 = np.cross(r3,r1)

/home/ee/mtech/eet172294/anaconda3/envs/gan/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """Entry point for launching an IPython kernel.
/home/ee/mtech/eet172294/anaconda3/envs/gan/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [195]:
cosTheta,sinTheta,r1,r2

(nan, nan, array([nan, nan, nan]), array([nan, nan, nan]))

### After Calculation of parameters, We double check by calculating 2d points from 3D and compare. 

In [196]:
W.shape,C.shape,m.shape

((6, 4), (6, 2), (3, 4))

Now We will calculate camaera coordinate (x and y) from world coordinate and estimated mateix M

The formula is                    
X = (m1 x Pi) / (m3 x Pi)              
Y = (m2 x Pi) / (m3 x Pi)

In [197]:
C_calculated = np.zeros((n, 2))
for i in range(n):
    C_calculated[i,0] = np.dot(m[0],W[i])/np.dot(m[2],W[i])
    C_calculated[i,1] = np.dot(m[1],W[i])/np.dot(m[2],W[i])
C_calculated = np.round(C_calculated)

# C_calculated

/home/ee/mtech/eet172294/anaconda3/envs/gan/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ee/mtech/eet172294/anaconda3/envs/gan/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


In [198]:
sqrd_error = (C-C_calculated)**2   # (x1-x2)^2 (y1-y2)^2
sqrd_error = sqrd_error[:,0]+sqrd_error[:,1]
sqrd_error = sqrd_error**(1/2) 
sqrd_error

array([nan, nan, nan, nan, nan, nan])

How much error

In [199]:
sqrd_error = np.sum(sqrd_error)/n
sqrd_error

nan